In [2]:
import psycopg2
import csv
import json
from datetime import datetime

def dbconnect(**params):
    return psycopg2.connect(**params)

params = {
    'host': "******",
    'port': "******",
    'database': "******",
    'user': "******",
    'password': "******"
}

conn = dbconnect(**params)

In [3]:
cur=conn.cursor()

In [4]:
def view_tables(cur):
    # Query to list all tables in the database
    cur.execute("""
        SELECT table_name
        FROM information_schema.tables
        WHERE table_schema = 'public'
        ORDER BY table_name;
    """)

    # Fetch and print all the table names
    tables = cur.fetchall()
    print("Tables in the database:")
    for table in tables:
        print(table[0])

view_tables(cur)

Tables in the database:
llmresponses
llms
tasks


In [5]:
conn.rollback()

In [6]:
def view_table_data(cur,table_name,data=False):
    # Replace 'your_table_name' with the name of the table you want to retrieve data from
    # Change this to the table you want to query

    # Execute a query to retrieve all rows from the table
    query = f"SELECT * FROM {table_name};"
    cur.execute(query)

    # Fetch all rows from the table
    rows = cur.fetchall()

    # Print the column names (optional)
    colnames = [desc[0] for desc in cur.description]
    print(f"Table:{table_name}\nColumns: {colnames}")

    if data:
        # Print each row
        print(f"Data from table '{table_name}':")
        for row in rows:
            print(row)

view_table_data(cur,"tasks")
view_table_data(cur,"llms")
view_table_data(cur,"llmresponses")

Table:tasks
Columns: ['taskid', 'question', 'expectedanswer', 'level', 'filename', 'filepath', 'annotations']
Table:llms
Columns: ['llmid', 'llmname', 'version', 'parameters']
Table:llmresponses
Columns: ['responseid', 'taskid', 'llmid', 'responsetext', 'isannotated', 'resultcategory', 'timestamp']


In [7]:
view_table_data(cur,"llms",True)
view_table_data(cur,"llmresponses",True)

Table:llms
Columns: ['llmid', 'llmname', 'version', 'parameters']
Data from table 'llms':
Table:llmresponses
Columns: ['responseid', 'taskid', 'llmid', 'responsetext', 'isannotated', 'resultcategory', 'timestamp']
Data from table 'llmresponses':


In [61]:
# DATA ALREADY INSERTED

# # Insert test data into 'llms' table
# llms_data = [
#     (1, 'GPT-3', 'v1', 175000000000),
#     (2, 'GPT-4', 'v2', 175000000000),
#     (3, 'ChatGPT', 'v1', 6800000000)
# ]

# cur.executemany(
#     "INSERT INTO llms (llmid, llmname, version, parameters) VALUES (%s, %s, %s, %s) ON CONFLICT DO NOTHING",
#     llms_data
# )

# conn.commit()
# print("Inserted data into llms table.")

Inserted data into llms table.


In [88]:
#DATA ALREADY INTO LLMRESPONSES

# # Fetch all taskids from the 'tasks' table
# cur.execute("SELECT taskid, expectedanswer FROM tasks")
# tasks = cur.fetchall()

# # Fetch all llmids from the 'llms' table
# cur.execute("SELECT llmid FROM llms")
# llms = cur.fetchall()

# # Define possible values for 'isannotated' and 'resultcategory'
# is_annotated_options = ['yes', 'no']
# result_category_options = ['AS IS', 'With Annotation', 'Helpless!']

# # Populate 'llmresponses' table with random data
# for i in range(101, 501):  # Insert 10 rows of test data
#     task = random.choice(tasks)  # Randomly select a task
#     llm = random.choice(llms)  # Randomly select an LLM

#     taskid = task[0]  # taskid from tasks table
#     llmid = llm[0]  # llmid from llms table
#     responsetext = task[1]  # Use expectedanswer from tasks as responsetext
#     isannotated = random.choice(is_annotated_options)  # Randomly select yes or no
#     resultcategory = random.choice(result_category_options)  # Randomly select category
#     timestamp = datetime.now()  # Current timestamp

#     cur.execute("""
#         INSERT INTO llmresponses (responseid, taskid, llmid, responsetext, isannotated, resultcategory, timestamp)
#         VALUES (%s, %s, %s, %s, %s, %s, %s)
#     """, (i, taskid, llmid, responsetext, isannotated, resultcategory, timestamp))

# conn.commit()
# print("Inserted data into llmresponses table.")


Inserted data into llmresponses table.


In [1]:
view_table_data(cur,"llms",True)
view_table_data(cur,"llmresponses",True)

NameError: name 'view_table_data' is not defined

# Metrics

In [10]:
def metric_cal(query):
    cur.execute(query)
    results=cur.fetchall()
    return results


## Accuracy Metrics <br>
### Overall Accuracy per LLM (As Is):<br>
Formula: (Number of "As Is" responses without annotation) / (Total number of tasks)


In [11]:
sql=''' SELECT
    LLMId,
    COUNT(CASE WHEN IsAnnotated = FALSE AND ResultCategory = 'As Is' THEN 1 END) * 100.0 / COUNT(DISTINCT TaskId) AS AccuracyPercentage
FROM
    LLMResponses
GROUP BY
    LLMId;'''

acc_per_llm= metric_cal(sql)
for row in acc_per_llm:
    print(f"LLM ID: {row[0]}, Overall Accuracy (As Is): {row[1]:.50f}%")

LLM ID: 1, Overall Accuracy (As Is): 0.00000000000000000000000000000000000000000000000000%
LLM ID: 2, Overall Accuracy (As Is): 0.00000000000000000000000000000000000000000000000000%
LLM ID: 3, Overall Accuracy (As Is): 0.00000000000000000000000000000000000000000000000000%


**Accuracy with Annotation:**

- **Formula:** (Number of "Correct with annotation" responses) / (Number of tasks that needed annotation)

In [12]:
# Query to calculate accuracy with annotation
query = """
    SELECT llmid,
           COUNT(CASE WHEN isannotated = True AND resultcategory = 'With Annotation' THEN 1 END) * 100.0 /
           COUNT(CASE WHEN isannotated = True THEN 1 END) AS accuracy_with_annotation
    FROM llmresponses
    GROUP BY llmid;
"""
acc_with_annotation=metric_cal(query)
# Display results
for row in acc_with_annotation:
    print(f"LLM ID: {row[0]}, Accuracy with Annotation: {row[1]:.2f}%")

LLM ID: 1, Accuracy with Annotation: 31.11%
LLM ID: 3, Accuracy with Annotation: 41.11%
LLM ID: 2, Accuracy with Annotation: 29.17%


## Improvement Rate

- **Formula:** (Number of tasks corrected after annotation) / (Number of tasks initially incorrect)

In [13]:
# Query to calculate improvement rate
query = """
    SELECT llmid,
           COUNT(CASE WHEN isannotated = 'yes' AND resultcategory = 'With Annotation' THEN 1 END) * 100.0 /
           COUNT(CASE WHEN isannotated = 'no' AND resultcategory != 'AS IS' THEN 1 END) AS improvement_rate
    FROM llmresponses
    GROUP BY llmid;
"""
impv_rate=metric_cal(query)

# Display results
for row in impv_rate:
    print(f"LLM ID: {row[0]}, Improvement Rate: {row[1]:.2f}%")

LLM ID: 1, Improvement Rate: 53.85%
LLM ID: 3, Improvement Rate: 72.55%
LLM ID: 2, Improvement Rate: 38.18%


**Failure Rate after Annotation**

- **Formula:** (Number of "Failed even after annotation" responses) / (Total number of tasks)

In [14]:
# Query to calculate failure rate after annotation
query = """
    SELECT llmid,
           COUNT(CASE WHEN isannotated = 'yes' AND resultcategory = 'Helpless!' THEN 1 END) * 100.0 /
           COUNT(DISTINCT taskid) AS failure_rate_after_annotation
    FROM llmresponses
    GROUP BY llmid;
"""
fail_rate_annotation=metric_cal(query)

# Display results
for row in fail_rate_annotation:
    print(f"LLM ID: {row[0]}, Failure Rate after Annotation: {row[1]:.2f}%")

LLM ID: 1, Failure Rate after Annotation: 30.36%
LLM ID: 2, Failure Rate after Annotation: 29.17%
LLM ID: 3, Failure Rate after Annotation: 24.30%


Performance by Task Level

In [15]:
# Query to calculate overall accuracy per LLM and task level
query = """
    SELECT llr.llmid, t.level,
           COUNT(CASE WHEN llr.isannotated = 'no' AND llr.resultcategory = 'AS IS' THEN 1 END) * 100.0 / 
           COUNT(DISTINCT llr.taskid) AS accuracy_percentage
    FROM llmresponses llr
    JOIN tasks t ON llr.taskid = t.taskid
    GROUP BY llr.llmid, t.level;
"""
Perf_task=metric_cal(query)
# Display results
for row in Perf_task:
    print(f"LLM ID: {row[0]}, Task Level: {row[1]}, Overall Accuracy (As Is): {row[2]:.2f}%")


LLM ID: 1, Task Level: 1, Overall Accuracy (As Is): 23.33%
LLM ID: 1, Task Level: 2, Overall Accuracy (As Is): 29.51%
LLM ID: 1, Task Level: 3, Overall Accuracy (As Is): 19.05%
LLM ID: 2, Task Level: 1, Overall Accuracy (As Is): 38.46%
LLM ID: 2, Task Level: 2, Overall Accuracy (As Is): 34.69%
LLM ID: 2, Task Level: 3, Overall Accuracy (As Is): 19.05%
LLM ID: 3, Task Level: 1, Overall Accuracy (As Is): 34.29%
LLM ID: 3, Task Level: 2, Overall Accuracy (As Is): 25.00%
LLM ID: 3, Task Level: 3, Overall Accuracy (As Is): 25.00%
